Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from zipfile import ZipFile
import wget
import keras
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions

Downloding Data

In [ ]:
#link = 'https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip'
#data = wget.download(link)

#with ZipFile('./dino-dragon.zip', 'r') as zobject:
#    zobject.extractall('./dino-dragon')

Loading Data

In [ ]:
path = "C:/Users/Godwin/Documents/Workflow/MLZoomcamp/dino-dragon/train/dragon/0d56640d-d68c-4f9f-9b6d-4de05201cc66.jpg"
img = load_img(path)

Build Model

In [ ]:
def make_model():
        
    input = keras.Input(shape =(150,150,3))

    conv_1 = keras.layers.Conv2D(32, kernel_size = (3,3), activation = 'relu')(input)
    pooling_1 = keras.layers.pooling.MaxPool2D(pool_size = (2,2))(conv_1)
    flatten = keras.layers.Flatten()(pooling_1)
    dense_1 = keras.layers.Dense(64, activation = 'relu')(flatten)
    output = keras.layers.Dense(1, activation = 'sigmoid')(dense_1)

    model = keras.Model(input, output)
    losses = keras.losses.BinaryCrossentropy()
    optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)

    model.compile(optimizer = optimizer, loss = losses, metrics = ['accuracy'])

    return model

Loading Data

In [ ]:
train_gen = ImageDataGenerator(rescale= 1./255)
train_data = train_gen.flow_from_directory("C:/Users/Godwin/Documents/Workflow/MLZoomcamp/dino-dragon/train/",class_mode='binary', target_size = (150,150), batch_size = 20, shuffle = True)

test_gen = ImageDataGenerator(rescale= 1./255)
test_data = test_gen.flow_from_directory('C:/Users/Godwin/Documents/Workflow/MLZoomcamp/dino-dragon/test/', class_mode = 'binary', target_size = (150,150), batch_size = 20,shuffle = True)

Training Model

In [ ]:
model = make_model()

history = model.fit(train_data, epochs=10, 
                    validation_data=test_data)

Visualizing Result

In [ ]:
plt.plot(history.history['accuracy'], label = 'Train')
plt.plot(history.history['val_accuracy'], label = 'Validation')

plt.xticks(np.arange(10))
plt.legend()

Calculating Median and Standard deviation

In [ ]:
median_val = round(np.median(history.history['accuracy']),2) #median of training accuracy
std = round(np.std(history.history['loss']),2) #standard deviation of training loss
print(median_val)
print(std)

Data Augumentation

In [ ]:
train_gen = ImageDataGenerator(rescale= 1./255,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )
train_data = train_gen.flow_from_directory('C:/Users/Godwin/Documents/Workflow/MLZoomcamp/dino-dragon/train/', class_mode = 'binary', target_size = (150,150), batch_size = 20, shuffle = True)

Training The model with Augumented data

In [ ]:
#Checkpointing
checkpoint = keras.callbacks.ModelCheckpoint(
                            'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
                              save_best_only = True,
                              monitor = 'val_accuracy',
                              mode = 'max')

In [ ]:
model = make_model()
history = model.fit(train_data, epochs=10,
                    validation_data=test_data,
                    callbacks = [checkpoint])

Visualizing Result

In [ ]:
plt.plot(history.history['accuracy'], label = 'Train')
plt.plot(history.history['val_accuracy'], label = 'Validation')

plt.xticks(np.arange(10))
plt.legend()

Calculating mean accucacy and loss 

In [ ]:
test_loss_mean = np.mean(history.history['val_loss']) #validation loss mean
accuracy = history.history['val_accuracy'] #validation acccuracy mean